![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to respond to natural language questions using RAG approach

**Note:** Please note that for the watsonx challenge, please consider running these notebooks locally on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of Retrieval Augumented Generation in watsonx.ai. It introduces commands for data retrieval, knowledge base building & querying, and model testing.

This notebook uses Python 3.10.

#### Objective

Use LLM model, Langchain and Milvus to create a Retrieval Augmented Generation (RAG) system. This will allow us to ask questions about our documents (that were not included in the training data), without fine-tunning the Large Language Model (LLM). When using RAG, if you are given a question, you first do a retrieval step to fetch any relevant documents from a special database, a vector database where these documents were indexed.

Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

##### Definitions:

* LLM - Large Language Model
* Langchain - a framework designed to simplify the creation of applications using LLMs
* Vector database - a database that organizes data through high-dimmensional vectors
* Milvus - vector database
* RAG - Retrieval Augmented Generation (see below more details about RAGs)

#### What is a Retrieval Augmented Generation (RAG) system?
Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.

The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

For the generator part, the obvious option is a LLM. In this Notebook we will use a quantized LLaMA v2 model, from the Kaggle Models collection.

The orchestration of the retriever and generator will be done using Langchain. A specialized function from Langchain allows us to create the receiver-generator in one line of code.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)


#### Data:

##### Knowledge base:
690,000 words worth of cleaned text from Wikipedia.

##### Train and Test datasets: 
There are three question files, one for each year of students: S08, S09, and S10.

The "questionanswerpairs.txt" files contain both the questions and answers. The columns in this file are as follows:

- ArticleTitle is the name of the Wikipedia article from which questions and answers initially came.
- Question is the question.
- Answer is the answer.
- DifficultyFromQuestioner is the prescribed difficulty rating for the question as given to the question-writer.
- DifficultyFromAnswerer is a difficulty rating assigned by the individual who evaluated and answered the question, which may differ from the difficulty in field 4.
- ArticleFile is the name of the file with the relevant article.

Questions that were judged to be poor were discarded from this data set.

There are frequently multiple lines with the same question, which appear if those questions were answered by multiple individuals. https://www.kaggle.com/rtatman/questionanswer-dataset


<a id="setup"></a>
##  Set up the environment



### Install and import dependecies

**Note:** For Windows environments, please remove `| tail -n 1` commands in the cell below.

In [1]:
#!pip install sentence_transformers | tail -n 1
#!pip install pandas | tail -n 1
#!pip install rouge_score | tail -n 1
#!pip install nltk | tail -n 1
#!pip install "ibm-watson-machine-learning>=1.0.312" | tail -n 1
#!pip install PyPDF2 | tail -n 1
#!pip install langchain | tail -n 1
#!pip install --upgrade pip | tail -n 1
#!pip install ibm-generative-ai | tail -n 1
#!pip install ipywidgets | tail -n 1
#!pip install ipywidgets widgetsnbextension pandas-profiling | tail -n 1
#!jupyter nbextension enable --py widgetsnbextension
#!python3 -m pip install tensorflow-macos | tail -n 1
#!python -m pip install tensorflow-metal | tail -n 1

In [2]:
# install milvus with [client] extras by pip
#!python3 -m pip install "milvus[client]" | tail -n 1
#!pip install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [3]:
import os
import pandas as pd
import glob
import numpy as np
import re
import string
import os
import zipfile
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
import requests
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)

from sentence_transformers import SentenceTransformer
from milvus import default_server
from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    connections,
    utility,
)
from sklearn.model_selection import train_test_split
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Milvus

from genai import Model
from genai.model import Credentials
from genai.schemas import GenerateParams
from dotenv import load_dotenv

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [4]:
load_dotenv()
BAM_API_URL = os.getenv("BAM_API_URL")
BAM_API_KEY = os.getenv("BAM_API_KEY")

In [5]:
credentials = Credentials(api_key=BAM_API_KEY, api_endpoint=BAM_API_URL)

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [6]:
try:
    project_id = os.getenv('PROJECT_ID')
except KeyError:
    pproject_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train and Test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [7]:
df_08 = pd.read_table('/Users/rrfsantos/Documents/GitHub/portifolio/Generative-AI/S08_question_answer_pairs.txt')
df_09 = pd.read_table('/Users/rrfsantos/Documents/GitHub/portifolio/Generative-AI/S09_question_answer_pairs.txt')
df_10 = pd.read_table('/Users/rrfsantos/Documents/GitHub/portifolio/Generative-AI/S10_question_answer_pairs.txt', 
                      encoding = 'windows-1252')  #engine = 'python', error_bad_lines = False)

print('-' * 15, df_08.shape, sep='\n')
print('-' * 15, df_09.shape, sep='\n')
print('-' * 15, df_10.shape, sep='\n')

---------------
(1715, 6)
---------------
(825, 6)
---------------
(1458, 6)


In [8]:
df = pd.concat([df_08, df_09, df_10], ignore_index=True)
df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleTitle'], axis = 1, inplace=True)
df.dropna(inplace=True)

print('-' * 15, df.shape, sep='\n')

---------------
(3420, 3)


In [9]:
# Limpar coluna "Answer"
def strip_last_punctuation(s):
  if s and s[-1] in string.punctuation:
    return s[:-1].strip()
  else:
    return s.strip()


df['answer_clean'] = df['Answer'].str.lower().map(strip_last_punctuation)

# Remove os dados faltantes da base de treino
df.dropna(inplace=True)

print('-' * 15, df.isna().sum(), sep='\n')

---------------
Question        0
Answer          0
ArticleFile     0
answer_clean    0
dtype: int64


In [10]:
df.drop(['Answer'], axis = 1, inplace=True)
df.drop_duplicates(subset=['answer_clean', 'Question'], keep='last', inplace = True)
print('-' * 15, df.shape, sep='\n')
print('-' * 15, df.isna().sum(), sep='\n')

---------------
(2783, 3)
---------------
Question        0
ArticleFile     0
answer_clean    0
dtype: int64


In [11]:
train_data, test_data = train_test_split(df, test_size=0.3)

## Build up knowledge base

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

We can generate dense vector representations using embedding models. In this notebook, we use [SentenceTransformers](https://www.google.com/search?client=safari&rls=en&q=sentencetransformers&ie=UTF-8&oe=UTF-8) [all-distilroberta-v1](https://huggingface.co/sentence-transformers/all-distilroberta-v1) to embed both the knowledge base passages and user queries.

A vector database is optimized for dense vector indexing and retrieval. This notebook uses [Milvus](https://milvus.io), a user-friendly open-source vector database, licensed under Apache 2.0, which offers good speed and performance with all-MiniLM-L6-v2 embedding model.

The size of each passage is limited by the embedding model's context window (which is 512 tokens for `all-distilroberta-v1`).

#### Load knowledge base documents

##### Cleaned text from Wikipedia

In [12]:
# Populate a dataframe with information about the docs available 

files = glob.glob('/Users/rrfsantos/Documents/GitHub/portifolio/Generative-AI/context/*.clean')

list_text = []
list_file = []

for file in files:
  with open(file, 'r', encoding = 'utf-8', errors='ignore') as f:
    text = f.read()
  list_text.append(re.sub(r'\n+', ' ', text).strip())
  list_file.append((os.path.basename(file)).split('.')[0])
df = pd.DataFrame({'title': list_file,'text': list_text})

missing_cols, missing_rows = ((df.isnull().sum(x) | df.eq(' ').sum(x))
        .loc[lambda x: x.gt(0)].index
        for x in (0, 1))

df.loc[missing_rows, missing_cols]
df['id'] = df.reset_index().index

In [13]:
documents = df.copy()

### Create an embedding function

Note that you can feed a custom embedding function to be used by Milvus. The performance of Milvus may differ depending on the embedding model used.

In [14]:
# Select a Sentence Transformer: https://www.sbert.net/docs/pretrained_models.html
embedding_model = SentenceTransformer("all-distilroberta-v1")

### Start Milvus

Start the Milvus embedded server

In [17]:
if default_server.running != True:
    default_server.start()
    print("Server should have now started")
else:
    default_server.stop()
    default_server.cleanup()
    default_server.start()
    print("Server is already running")

Server is already running


Establish a connection with the embedded server and print its version information.

In [18]:
connections.connect(host="localhost", port=default_server.listen_port)
print(utility.get_server_version())

v2.3.3-lite


Define the collection

In [19]:
COLLECTION_NAME = "wikipedia_collection"
INDEX_NAME = "wikipedia_index"

In [20]:
# Run if you want to drop your old data
try:
    utility.drop_collection(COLLECTION_NAME)
    print("Collection has been deleted")
except:  # noqa: E722
    pass

Collection has been deleted


In [21]:
id = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
    auto_id=True,
)

text = FieldSchema(
    name="text",
    dtype=DataType.VARCHAR,
    max_length=6000,
)

text_vector = FieldSchema(name="text_vector", dtype=DataType.FLOAT_VECTOR, dim=768)

qid = FieldSchema(name="qid", dtype=DataType.INT64)

title = FieldSchema(
    name="title",
    dtype=DataType.VARCHAR,
    max_length=5120,
)

schema = CollectionSchema(
    fields=[id, text, text_vector, qid, title],
    description="SIEM vector store",
    enable_dynamic_field=True,
)

collection = Collection(
    name=COLLECTION_NAME, schema=schema, using="default", shards_num=2
)

### Embed and index documents with Milvus

**Note: Could take several minutes if you don't have pre-built indices**


Prepare collection

In [22]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=50, length_function=len, add_start_index=False
)


def split_and_prepare_document_new(qid: str, title: str, text: str):
    split_text = text_splitter.split_text(text)
    ids = [qid] * len(split_text)
    titles = [title] * len(split_text)
    embeddings = [
        embedding_model.encode(xc, show_progress_bar=False) for xc in split_text
    ]
    return split_text, ids, titles, embeddings


def process_batch(document_list):
    batch_results = []

    for id, title, text in zip(
        document_list["id"].values.tolist(),
        document_list["title"].values.tolist(),
        document_list["text"].values.tolist(),
    ):
        for sub_text, sub_id, sub_title, sub_embedding in zip(
            *split_and_prepare_document_new(id, title, text)
        ):
            batch_results.append(tuple((sub_id, sub_title, sub_text, sub_embedding)))
    return batch_results

In [23]:
batch_size = 10
processed_docs = []
cache_filename = '/Users/rrfsantos/Documents/GitHub/portifolio/prepared-docs.pkl'
allow_cache = True

if allow_cache and os.path.isfile(cache_filename):
    os.remove(cache_filename)

for i in tqdm(range(0, len(documents), batch_size), desc="Processing Documents in Batches"):
    # find end of batch
    i_end = min(i + batch_size, len(documents))
    documents_batch = documents[i:i_end]

    # Process the batch
    processed = process_batch(documents_batch)
    processed_docs.extend(processed)

with open(cache_filename, "wb") as f:
    pickle.dump(processed_docs, f)

print("Processed docs saved to pickle checkpoint")

Processing Documents in Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Processed docs saved to pickle checkpoint


Insert the embeddings, texts, titles and documents id's in collection.

In [24]:
if default_server.running:
    collection = Collection(COLLECTION_NAME)
    error  = []
    batch_size = 1000
    for i in tqdm(
        range(0, len(processed_docs), batch_size),
        desc="Inserting documents batches to Milvus VectorDB",
    ):
        # find end of batch
        i_end = min(i + batch_size, len(processed_docs))
        id_l, title_l, text_l, embed_l = list(zip(*processed_docs[i:i_end]))

        data_to_insert = [text_l, embed_l, id_l, title_l]
        try:
            collection.insert(data_to_insert)
        except Exception as ex:
            print(f"Failed to insert: {ex}")
            error.append(title_l)
else:
    print("Milvus server is not running! Rerun related notebook cells.")

Inserting documents batches to Milvus VectorDB:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
NLIST_SIZE = 1024

index_params = {
    "metric_type": "COSINE",
    "index_type": "HNSW",
    "params": {"nlist": NLIST_SIZE},
    "M": 16,
    "efConstruction": 200,
}

collection.create_index(field_name="text_vector", index_params=index_params)

print("Collection index has been successfully created!")

Collection index has been successfully created!


<a id="models"></a>
## Foundation Models on Watsonx

In [26]:
load_dotenv()
BAM_API_URL = os.getenv("BAM_API_URL")
BAM_API_KEY = os.getenv("BAM_API_KEY")

In [27]:
# get the list of supported models from the API
models = Model.models(credentials=credentials)

model_ids = []
for model_n in models:
    print(model_n.id)

salesforce/codegen2-16b
codellama/codellama-34b-instruct
tiiuae/falcon-180b
tiiuae/falcon-40b
ibm/falcon-40b-8lang-instruct
google/flan-t5-xl
google/flan-t5-xxl
google/flan-ul2
eleutherai/gpt-neox-20b
ibm/granite-13b-chat-v1
ibm/granite-13b-chat-v2
ibm/granite-13b-instruct-v1
ibm/granite-13b-instruct-v2
ibm/granite-20b-code-instruct-v1
ibm/granite-3b-code-plus-v1
elyza/japanese-llama-2-7b-fast
elyza/japanese-llama-2-7b-instruct
meta-llama/llama-2-13b
meta-llama/llama-2-13b-chat
meta-llama/llama-2-13b-chat-beam
meta-llama/llama-2-70b
meta-llama/llama-2-70b-chat
thebloke/llama-2-70b-chat-gptq
meta-llama/llama-2-7b
meta-llama/llama-2-7b-chat
mosaicml/mpt-30b
ibm/mpt-7b-instruct
bigscience/mt0-xxl
defog/sqlcoder-34b-alpha
bigcode/starcoder
flan-t5-xl-mpt-HrlayZEh-2023-10-25-18-15-34


In [42]:
# select generative model to use
model_id = "thebloke/llama-2-70b-chat-gptq"

# Iterate over the "results" list to find the matching model ID
for model_n in models:
    if model_n.id == model_id:
        model_token_limit = model_n.token_limit
        print(f"Model was found, it's token limit is {model_token_limit}.")
        break
else:
    print("Model was not found, pick a different one!")
    model_token_limit = None

Model was found, it's token limit is 4096.


In [48]:
# set-up inference parameters
params = GenerateParams(decoding_method="greedy",
                        max_new_tokens=500,
                        min_new_tokens=1,
                        repetition_penalty=1,
                        temperature=0.5,
                        top_k=50,
                        top_p=1)

model = Model(model=model_id, credentials=credentials, params=params)

The input token limit depends on the selected generative model's max sequence length. The total input tokens in the RAG prompt should not exceed the model's max sequence length minus the number of desired output tokens. The choice of the number of paragraphs to retrieve as context impacts the number tokens in the prompt.

In [49]:
# For setting the input token limit we subtract the max_new_tokens (to be generated) and -1 from the model_token_limit
input_token_limit = model_token_limit - params.max_new_tokens - 1
print(f"Input token limit: {input_token_limit}")

Input token limit: 3595


<a id="predict"></a>
## Generate a retrieval-augmented response to a question

### Feed the context and the examples to `watsonx.ai` model.

##### Feed the context and the examples to `genai` model

In [50]:
# Token counting function
def token_count(doc):
    return model.tokenize([doc])[0].token_count

`prompt_template` is a function to create a prompt from the given context and question. Changing the prompt will sometimes result in much more appropriate answers (or it may degrade the quality significantly). The prompt template below is most appropriate for short-form extractive use cases.

`make_prompt` includes a script to truncate the context length provided as an input in case the total token inputs exceed the model's limit. The paragraphs with the largest distance are truncated first. This functionality is helpful in case the embedded passages are not of the same size.

In [51]:
examples = train_data.sample(n=10)

In [52]:
few_shot_example=[]
few_shot_examples=[]

for index, row in examples.iterrows():
    input = row.Question
    output = row.answer_clean
    few_shot_example.append(f"input: {input}\noutput: {output}")
few_shot_examples='\n\n\n'.join(few_shot_example)

In [53]:
def prompt_template(context, few_shot_examples, question_text):
    return ('''Below is an instruction that describes a task, paired with an input that provides further context and questions and answer examples.

### Input:

Based on the context and the examples, write one Response that appropriately answer the Question below.
Provide your output in a maximum of 3 sentences.

Context:'''
            
+ "\n\n"
+ f"{context}\n\n" 
+ "##\n\n"
+ "Examples:\n\n"
+ f"{few_shot_examples}\n\n"
+ f"Question: {question_text}\n\n"

"### Response:"
)

In [54]:
def make_prompt(question, few_shot_examples, input_token_limit):

    ### Create question embedding
    question_embeddings = embedding_model.encode(question)

    ### Collect the context in vetctordb
    
    search_params = {"params": {"nprobe": 10, "radius": 0.60}, "metric_type": "COSINE"} # radius ---> minimum similarity

    results = collection.search(
       data=[question_embeddings],
       anns_field="text_vector",
       limit=2,
       output_fields=["id","text"],  # name of the field to retrieve from the search result
       param=search_params,
       round_decimal=-1
    )
    
    print(f"Found {len(results[0])} results in the collection.")

    documents = []
    for raw_result in results:
        for result in raw_result:
            documents.append(result.entity.get("text"))
            #print("=========")
            #print("Paragraph : ", result.entity.get("text"))
            print("Distance : ", result.distance)

    context = "\n\n\n".join(documents)

    prompt = prompt_template(context, few_shot_examples, question)

    prompt_token_count = token_count(prompt)

    if prompt_token_count <= input_token_limit:
        return prompt

    print("exceeded input token limit, truncating context", prompt_token_count)
    
    # documents with the lower distance scores are included in the truncated context first
    distances = results[0].distances
    sorted_indices = sorted(range(len(distances)), key=lambda k: distances[k])

    truncated_context = ""
    token_count_so_far = 0
    i = 0

    while token_count_so_far <= input_token_limit and i < len(sorted_indices):
        doc_index = sorted_indices[i]
        document = documents[doc_index]
        doc_token_count = token_count(document) + token_count(few_shot_examples)

        if token_count_so_far + doc_token_count <= input_token_limit:
            truncated_context += document + "\n\n\n"
            token_count_so_far += doc_token_count
        else:
            remaining_tokens = input_token_limit - token_count_so_far
            truncated_context += document[:remaining_tokens]
            break

        i += 1

    return prompt_template(truncated_context, few_shot_examples, question)

In [55]:
%time
questions = test_data.sample(n=20)
questions = questions['Question'].tolist()
collection = Collection(COLLECTION_NAME)      # Get the existing collection.
collection.load() 
prompt_texts = []
for question in questions:
    prompt_text = make_prompt(question, few_shot_examples, input_token_limit)
    prompt_texts.append(prompt_text)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


Found 1 results in the collection.
Distance :  0.6147989630699158
Found 0 results in the collection.
Found 0 results in the collection.
Found 0 results in the collection.
Found 2 results in the collection.
Distance :  0.6273016929626465
Distance :  0.6147995591163635
Found 2 results in the collection.
Distance :  0.6437525153160095
Distance :  0.6328293681144714
Found 2 results in the collection.
Distance :  0.666768491268158
Distance :  0.6583305597305298
Found 0 results in the collection.
Found 0 results in the collection.
Found 2 results in the collection.
Distance :  0.6847279071807861
Distance :  0.6610898971557617
Found 1 results in the collection.
Distance :  0.6345176100730896
Found 1 results in the collection.
Distance :  0.6066610813140869
Found 0 results in the collection.
Found 0 results in the collection.
Found 0 results in the collection.
Found 0 results in the collection.
Found 1 results in the collection.
Distance :  0.601598858833313
Found 2 results in the collection.


Generate responses

In [56]:
%time
answers = []
for response in model.generate(prompt_texts):
    answers.append(response.generated_text)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 9.06 µs


In [57]:
print(prompt_texts[0])

Below is an instruction that describes a task, paired with an input that provides further context and questions and answer examples.

### Input:

Based on the context and the examples, write one Response that appropriately answer the Question below.
Provide your output in a maximum of 3 sentences.

Context:

elephant Elephants (Elephantidae) are a family in the order Proboscidea in the class Mammalia. They were once classified along with other thick skinned animals in a now invalid order, Pachydermata. There are three living species: the African Bush Elephant, the African Forest Elephant (until recently known collectively as the African Elephant), and the Asian Elephant (also known as the Indian Elephant). Other species have become extinct since the last ice age, which ended about 10,000 years ago, the Mammoth being the most well-known of these. The word "elephant" has its origins in the Greek ἐλέφας, meaning "ivory" or "elephant". Elephants are mammals, and the largest land animals al

In [58]:
df_result = pd.DataFrame({'Question': questions, 'Answer': answers})

In [59]:
print(df_result.Question[0])
print(df_result.Answer[0])

Where was the largest elephant ever recorded shot?


The largest elephant ever recorded was shot in Angola in 1956.


In [60]:
#default_server.stop()